In [26]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append("../../utils/")

from GraphAPI import GraphCreator
from graph_helpers import *

%aimport GraphAPI
%aimport graph_helpers

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Generating Graph from Entry Point

1. We initialize our GraphCreator class and check how many new nodes we will need to query. 

In [27]:
gc = GraphCreator("Sonata cycle")
print("Number of Links to Search:", len(gc.next_links))
print(gc.primary_nodes)

Number of Links to Search: 147
{}


2. We query all the nodes linked to/from the entry point (expand our network one level for each node).

In [28]:
gc.expand_network(group_size=2, timeout=5, log_progress=False)

3. Since some nodes will likely have linked to articles through a redirect link, we need to traverse our graph and ensure that all redirects are assigned to the correct nodes. Once all redirects have been dealt with, we remove any old redirect node. 

In [29]:
gc.redraw_redirects()

4. Edges are weighted by how many categories two connected nodes have in common. Once we have all our nodes, and we have dealt with redirects, we can add edge weights for our entire graph. 

In [30]:
gc.update_edge_weights()
gc.get_edge_weights().head()

,source_node,target_node,edge_weight
0,"Six Sonatas for Violin and Harpsichord, BWV 10...",Schubert's compositions for violin and piano,5
1,Sonata for two clarinets,Sonata for clarinet and bassoon,5
2,Schubert's compositions for violin and piano,"Six Sonatas for Violin and Harpsichord, BWV 10...",5
3,Cello Sonata (Poulenc),Sonata for clarinet and bassoon,5
4,Sonata for clarinet and bassoon,Cello Sonata (Poulenc),5


# Getting Our Feature Set

There are two options when generating the feature set:

1. we can generate a standard feature set with only the features themselves. To do this, have the `rank` parameter set to `False`.
2. We can generate a ranked feature set (set `rank` equal to `True`). For each parameter, this will rank them in order of _best_ to _worst_ (this could be ascending or descending, depending on the context of the feature).

After running `get_features_df`, the feature set will be saved in the GraphCreator instance as `feature_df`

In [34]:
features_df = gc.get_features_df(rank=False)

In [35]:
features_df.sort_values("shared_neighbors_with_entry_score", ascending=False)

,node,degree,category_matches_with_source,in_edges,out_edges,shared_neighbors_with_entry_score,centrality,page_rank,adjusted_reciprocity,shortest_path_length_from_entry,shortest_path_length_to_entry,primary_link
77,Sonata cycle,147,1,69.0,78.0,1.000000,0.085738,0.000866,130.0,0.0,0.0,0
78,Piano sonatas (Chopin),141,0,71.0,70.0,0.717391,0.081460,0.000766,130.0,1.0,1.0,0
74,Sonata theory,161,0,78.0,83.0,0.673267,0.097959,0.001050,148.0,1.0,1.0,0
75,List of solo keyboard sonatas by Domenico Scar...,159,0,84.0,75.0,0.628571,0.099888,0.001264,140.0,1.0,1.0,0
76,Cello Sonatas (Vivaldi),157,0,67.0,90.0,0.611111,0.132349,0.000999,130.0,1.0,1.0,0
80,International Standard Book Number,50,0,NaN,NaN,0.609756,0.052594,0.001584,0.0,2.0,NaN,0
73,Sonata da camera,165,0,82.0,83.0,0.596491,0.098264,0.001045,130.0,1.0,1.0,0
72,Cello Sonata (Chopin),182,0,97.0,85.0,0.532258,0.145200,0.001582,148.0,1.0,1.0,0
70,Horn Sonata (Beethoven),192,0,89.0,103.0,0.531746,0.134734,0.001249,162.0,1.0,1.0,0
69,Cello Sonata (Grieg),203,0,93.0,110.0,0.511450,0.147432,0.001421,174.0,1.0,1.0,0


## Similarity Rank

Two articles are more similar the more categories they share and the closer they are to each other. 

In [ ]:
gc.rank_similarity()
gc.features_df[[
    "node", 
    "category_matches_with_source", 
    "primary_link", 
    "shortest_path_length_from_entry",
    "shortest_path_length_to_entry",
    "centrality",
    "similarity_rank", 
]].sort_values(["similarity_rank", "centrality"], ascending=False).head(50)
# gc.features_df[gc.features_df.category_matches_with_source == 1]

In [ ]:
gc.intro_nodes

In [25]:
test = list(set(nx.all_neighbors(gc.graph, gc.entry)))
# len({x : True for x in test + test})
len(set(test))
# len(set(test + test))
# len(test) + len(test) - len(set(test + test))

82

# Basic Plotting

In [ ]:
sns.pairplot(features_df)

In [ ]:
sns.heatmap(features_df.corr())

# Intro Node Values

In [ ]:
intro_nodes = gc.intro_nodes
intro_nodes_df = None
for node in intro_nodes:
    if df is None:
        intro_nodes_df = pd.DataFrame(features_df[features_df.node == node])
    else:
        intro_nodes_df = pd.concat([intro_nodes_df, pd.DataFrame(features_df[features_df.node == node])])
    
intro_nodes_df

In [ ]:
sns.pairplot(intro_nodes_df, hue="node")